### scrapy
- 비동기 방식으로 데이터를 수집 > 데이터 수집 속도가 빠름

In [1]:
import scrapy, requests
from scrapy.http import TextResponse

In [2]:
# gmarket 베스트 200 상품 데이터 수집

In [3]:
# 1. 스크레피 프로젝트 생성

In [7]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'C:\Users\Wslee\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\Wslee\Desktop\github\aivle\3. web crawling\day3\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [8]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 00000054 D3A3:08A0입니다.
C:\USERS\WSLEE\DESKTOP\GITHUB\AIVLE\3. WEB CRAWLING\DAY3\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [9]:
# items.py : 수집할 데이터의 구조 정의
# middlewares.py : 데이터를 수집할때 headers 정보와 같은 내용을 설정
# pipelines.py : 데이터를 수집한 후에 코드를 실행 정의
# settings.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙 지킬지 말지
# spiders : 디렉토리 : 크롤링 절차 정의

In [ ]:
# spiders 실행
# items 모델, 모양데로 데이터 수집
# pipelines 수집된 데이터를 데이터베이스에 저장

In [4]:
# 2. xpath 찾기 : 링크, 상세 페이지의 데이터

In [ ]:
url = "http://corners.gmarket.co.kr/Bestsellers"

In [11]:
# 링크 데이터 수집

In [15]:
request = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [21]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953127173933095',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953127173933095']

In [ ]:
# 상세 페이지 데이터 수집

In [23]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [40]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [5]:
# 3. items.py : 코드 작성 : model(데이터의 구조: 수집할 데이터의 칼럼을 정의)

In [41]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Writing gmarket/gmarket/spiders/spider.py


In [6]:
# 4. spider.py : 코드 작성 : 크롤링 절차 정의

In [43]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"] # 최초 request하는 url 알아서 만들어준 url
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. 스크레피 프로젝트 실행
# gmarket 디렉토리
# scrapy crawl GMB -o items.csv

In [44]:
%pwd

'C:\\Users\\Wslee\\Desktop\\github\\aivle\\3. web crawling\\day3'

In [1]:
# cd C:\\Users\\Wslee\\Desktop\\github\\aivle\\3. web crawling\\day3\gmarket
# scrapy start project 한 폴더 안에서 실행 가능
# scrapy crawl GMB -o items.csv

C:\Users\Wslee\Desktop\github\aivle\3. web crawling\day3


In [3]:
pd.read_csv("items.csv").tail(2)

,link,price,title
18,http://item.gmarket.co.kr/Item?goodscode=25204...,"28,800",(롯데백화점)남여공용 디스럽터 테이피 테잎 _ FS1HTB1091X
19,http://item.gmarket.co.kr/Item?goodscode=22664...,"3,900",3900원 균일가 시즌오프 블라우스/티셔츠/원피스/팬츠 外


In [ ]:
# settings.py
# Obey robots.txt rules
# ROBOTSTXT_OBEY = True # False 해야 naver 같이 막아놓은 사이트도 crawling 가능